# Dataset về đề xuất nhạc trong ô tô có tính đến ngữ cảnh


### Import các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Thiết lập hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('default')
sns.set_palette('husl')

### Đọc dữ liệu từ file Excel

In [2]:
# Đọc file Excel từ nhiều sheet như trong exploration notebook
con_rat = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=0).rename(columns={' Rating': 'label', 'UserID': 'user', 'ItemID': 'item'})
con_fac = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=1)
mus_trk = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=2).rename(columns={' category_id': 'category_id'})
mus_cat = pd.read_excel('./Data_InCarMusic.xlsx', sheet_name=3, header=None).rename(columns={0:'genre_id', 1:'genre'})

# Tạo dictionary cho genre mapping
mus_cat_dict = mus_cat.set_index('genre_id').genre.str.split(' ').str[0].to_dict()
mus_trk['genre'] = mus_trk.category_id.apply(lambda x: mus_cat_dict.get(x))

# Sử dụng con_rat làm dataframe chính
df = con_rat.copy()

print(f'\nKích thước dữ liệu: {df.shape[0]} hàng x {df.shape[1]} cột')
print(f'Các cột: {list(df.columns)}')



Kích thước dữ liệu: 4012 hàng x 11 cột
Các cột: ['user', 'item', 'label', 'DrivingStyle', 'landscape', 'mood', 'naturalphenomena ', 'RoadType', 'sleepiness', 'trafficConditions', 'weather']


## Khám phá dữ liệu
### Cấu trúc tổng quan

In [3]:
# Hiển thị 5 dòng đầu tiên
print(df.head())

   user  item  label DrivingStyle landscape mood naturalphenomena  RoadType  \
0  1001   715      2          NaN       NaN  NaN               NaN      NaN   
1  1001   267      4          NaN       NaN  NaN               NaN      NaN   
2  1001   294      2          NaN       NaN  NaN               NaN      NaN   
3  1001   259      4          NaN       NaN  NaN               NaN      NaN   
4  1001   674      2          NaN       NaN  NaN               NaN      NaN   

  sleepiness trafficConditions  weather  
0        NaN               NaN    sunny  
1        NaN               NaN    sunny  
2        NaN               NaN    sunny  
3        NaN               NaN  snowing  
4        NaN               NaN    rainy  


In [4]:
# Thông tin về các cột
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4012 entries, 0 to 4011
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user               4012 non-null   int64 
 1   item               4012 non-null   int64 
 2   label              4012 non-null   int64 
 3   DrivingStyle       353 non-null    object
 4   landscape          505 non-null    object
 5   mood               406 non-null    object
 6   naturalphenomena   396 non-null    object
 7   RoadType           396 non-null    object
 8   sleepiness         157 non-null    object
 9   trafficConditions  399 non-null    object
 10  weather            397 non-null    object
dtypes: int64(3), object(8)
memory usage: 344.9+ KB
None


### Thống kê mô tả dữ liệu

#### Thống kê cho các cột số

In [13]:
 numeric_df = df.select_dtypes(include=["number"])
 print(numeric_df.describe())


              user         item        label
count  4012.000000  4012.000000  4012.000000
mean   1019.090728   555.172483     0.257976
std      10.916548   216.942497     0.437575
min    1001.000000   248.000000     0.000000
25%    1009.000000   280.000000     0.000000
50%    1019.000000   692.000000     0.000000
75%    1029.000000   729.000000     1.000000
max    1042.000000   762.000000     1.000000


#### Thống kê cho các cột phân loại (categorical)

In [7]:
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f'\n--- {col} ---')
    print(df[col].value_counts())
    print(f'Số lượng giá trị duy nhất: {df[col].nunique()}')


--- DrivingStyle ---
DrivingStyle
relaxed driving    181
sport driving      172
Name: count, dtype: int64
Số lượng giá trị duy nhất: 2

--- landscape ---
landscape
mountains       140
coast line      125
country side    122
urban           118
Name: count, dtype: int64
Số lượng giá trị duy nhất: 4

--- mood ---
mood
active    104
sad       102
lazy      101
happy      99
Name: count, dtype: int64
Số lượng giá trị duy nhất: 4

--- naturalphenomena  ---
naturalphenomena 
morning      103
day time     101
afternoon     97
night         95
Name: count, dtype: int64
Số lượng giá trị duy nhất: 4

--- RoadType ---
RoadType
highway       138
serpentine    132
city          126
Name: count, dtype: int64
Số lượng giá trị duy nhất: 3

--- sleepiness ---
sleepiness
sleepy    85
awake     72
Name: count, dtype: int64
Số lượng giá trị duy nhất: 2

--- trafficConditions ---
trafficConditions
lots of cars    140
traffic jam     133
free road       126
Name: count, dtype: int64
Số lượng giá trị duy nh

### Kiểm tra giá trị thiếu (Missing Values)

In [9]:
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Số lượng thiếu': missing_values, 'Phần trăm (%)': missing_percent})
missing_df = missing_df[missing_df['Số lượng thiếu'] > 0].sort_values('Số lượng thiếu', ascending=False)
print(missing_df)

                   Số lượng thiếu  Phần trăm (%)
sleepiness                   3855      96.086740
DrivingStyle                 3659      91.201396
RoadType                     3616      90.129611
naturalphenomena             3616      90.129611
weather                      3615      90.104686
trafficConditions            3613      90.054835
mood                         3606      89.880359
landscape                    3507      87.412762


## Tiền xử lý dữ liệu

In [10]:
# Tìm các cột context (categorical)
cat_cols = df.columns[df.dtypes == 'object']
print("Các cột context:", list(cat_cols))

# In ra các giá trị unique cho mỗi cột context
print("\nCác giá trị unique cho mỗi cột context:")
for c in cat_cols:
    unique_vals = df[c].unique()
    print(f'{str(unique_vals):<55} #{len(unique_vals)}')

# Tạo binary label như trong exploration
df['label'] = np.select([df.label > 3, df.label <= 3], [1, 0])

# Xử lý missing values - thay thế NaN bằng 'Unknown' thay vì loại bỏ
for col in cat_cols:
    df[col] = df[col].fillna('Unknown')

print(f"\nSau khi xử lý missing values:")
print(f"Số dòng: {len(df)}")
print(f"Số user: {df['user'].nunique()}")
print(f"Số item: {df['item'].nunique()}")


Các cột context: ['DrivingStyle', 'landscape', 'mood', 'naturalphenomena ', 'RoadType', 'sleepiness', 'trafficConditions', 'weather']

Các giá trị unique cho mỗi cột context:
[nan 'relaxed driving' 'sport driving']                 #3
[nan 'urban' 'mountains' 'country side' 'coast line']   #5
[nan 'sad' 'lazy' 'active' 'happy']                     #5
[nan 'night' 'morning' 'day time' 'afternoon']          #5
[nan 'city' 'serpentine' 'highway']                     #4
[nan 'sleepy' 'awake']                                  #3
[nan 'traffic jam' 'lots of cars' 'free road']          #4
['sunny' 'snowing' 'rainy' 'cloudy' nan]                #5

Sau khi xử lý missing values:
Số dòng: 4012
Số user: 42
Số item: 139


In [11]:
# Xử lý context với one-hot encoding như trong exploration notebook
# Tạo one-hot encoding cho các cột context
features_oh = pd.get_dummies(df[cat_cols], dummy_na=True)

# Tạo context string từ one-hot encoding
context_series = (features_oh * features_oh.columns).sum(axis=1)
context_series.rename('context', inplace=True)

# Tạo dataframe cuối cùng với context
feat_data = pd.concat([df[['user', 'item', 'label']], context_series], axis=1)

print("Dataframe sau khi xử lý context:")
print(feat_data.head())
print(f"\nSố lượng context unique: {feat_data['context'].nunique()}")
print(f"Các context mẫu: {feat_data['context'].unique()[:10]}")


Dataframe sau khi xử lý context:
   user  item  label                                            context
0  1001   715      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
1  1001   267      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
2  1001   294      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
3  1001   259      1  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...
4  1001   674      0  DrivingStyle_Unknownlandscape_Unknownmood_Unkn...

Số lượng context unique: 27
Các context mẫu: ['DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_sunny'
 'DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_snowing'
 'DrivingStyle_Unknownlandscape_Unknownmood_Unknownnaturalphenomena _UnknownRoadType_Unknownsleepiness_UnknowntrafficConditions_Unknownweather_rainy'
 'DrivingStyle_Unknownlandscape_Unkno

In [12]:
# Xuất dữ liệu 
import os
out_dir = 'output_carskit'
os.makedirs(out_dir, exist_ok=True)

# Xuất ratings với context (cho LightFM)
ratings_ctx_path = os.path.join(out_dir, 'ratings_with_context.csv')
feat_data.to_csv(ratings_ctx_path, index=False, encoding='utf-8')

# Xuất ratings chỉ có user, item, label (cho collaborative filtering)
ratings_only = feat_data[['user', 'item', 'label']].copy()
ratings_only_path = os.path.join(out_dir, 'ratings_only.csv')
ratings_only.to_csv(ratings_only_path, index=False, encoding='utf-8')

# Xuất thông tin item với genre (cho item features)
item_features = mus_trk[['id', 'genre']].rename(columns={'id': 'item'})
item_features_path = os.path.join(out_dir, 'item_features.csv')
item_features.to_csv(item_features_path, index=False, encoding='utf-8')

print("Đã lưu các file:")
print(f" - {ratings_ctx_path}")
print(f" - {ratings_only_path}")
print(f" - {item_features_path}")

# Thống kê cuối cùng
print(f"\nThống kê cuối cùng:")
print(f"Số user: {feat_data['user'].nunique()}")
print(f"Số item: {feat_data['item'].nunique()}")
print(f"Số interaction: {len(feat_data)}")
print(f"Số context unique: {feat_data['context'].nunique()}")
print(f"Tỷ lệ positive (label=1): {feat_data['label'].mean():.3f}")


Đã lưu các file:
 - output_carskit\ratings_with_context.csv
 - output_carskit\ratings_only.csv
 - output_carskit\item_features.csv

Thống kê cuối cùng:
Số user: 42
Số item: 139
Số interaction: 4012
Số context unique: 27
Tỷ lệ positive (label=1): 0.258
